In [ ]:
import sqlite3
import pandas as pd
import csv
import zipfile, io
import chardet
from tqdm import tqdm
import json
import srt

dbpath = "E:\Technologie\Master\Webm\opensubtitles.org.Actually.Open.Edition.2022.07.25\opensubs.db"
linked_subtitles = "E:\Technologie\Master\Webm\subtitles_all_linked.pkl"

#output_file = "E:\Technologie\Master\Webm\subtitles_final.pkl" #intermediate file, no longer used

output_folder = "E:\Technologie\Master\Webm\srt\\"

In [ ]:
# load the dataframe from the pickle file
#df = pd.read_pickle("E:\Technologie\Master\Webm\subtitles_all.pkl")

In [ ]:
df_index = pd.read_pickle(linked_subtitles)

In [ ]:
ids = df_index["IDSubtitle"].tolist()

In [ ]:
# Connect to database
conn = sqlite3.connect(dbpath)
c = conn.cursor()

# find table names
c.execute("SELECT name FROM sqlite_master WHERE type='table';")

# print all table names
# print(c.fetchall())

# find column names
c.execute("SELECT * FROM subz LIMIT 1")
# print([description[0] for description in c.description])

#find primary key
c.execute("SELECT sql FROM sqlite_master WHERE name='subz' AND type='table';")
# print(c.fetchall())

data = []

length = len(ids)

for row in tqdm(c.execute(f'SELECT * FROM subz WHERE num in {tuple(ids)}'),total=length):
    zip_data = row[2]

    #print(row)
    # Create a file-like object from the binary data
    zip_file = io.BytesIO(zip_data)

    # This can now be used like a normal zipfile
    with zipfile.ZipFile(zip_file) as myzip:
        if len([zip_info.filename for zip_info in myzip.infolist() if zip_info.filename.endswith(".srt")]) == 1: # max 1 srt file
            for zip_info in myzip.infolist():

                # keep only the files with the extension "srt"
                if zip_info.filename.endswith(".srt"):

                    with myzip.open(zip_info) as myfile:

                        file_content = myfile.read()

                        # # Detect its encoding
                        #detected = chardet.detect(file_content)
                        # print(detected)
                        #if detected["encoding"] in ["ISO-8859-1","Windows-1254","ascii"]:
                        data.append([row[0],file_content.decode("ascii", "ignore")])


# Close connection
conn.close()

In [ ]:
len(data)

In [ ]:
# for d in data:
#     # Detect its encoding
#     detected = chardet.detect(d[1])
#     print(detected)
#     if detected["encoding"] in ["ascii"]:
#         # decode the bytes object
#         d[1] = d[1].decode("ascii")
#         #print(d)

In [ ]:
df = pd.DataFrame(data, columns=['id', 'subtitles'])
df

In [ ]:
# merge the two dataframes on the column "IDSubtitle"
df_index['IDSubtitle'] = pd.to_numeric(df_index['IDSubtitle'], errors='coerce')
df_merged = pd.merge(df, df_index, left_on='id', right_on='IDSubtitle', how='inner')
df_merged = df_merged[["uuid","MovieName","MovieYear","subtitles"]]
df_merged

In [ ]:
# pickling the dataframe
#df_merged.to_pickle(output_file)

In [ ]:
# Generate the json files

length = len(df_merged)
for i, subtitles in tqdm(df_merged.iterrows(), total=length):
    try:
        generator = srt.parse(df_merged.loc[i, 'subtitles'])
        subtitles = list(generator)
        
        output_subtitles = []
        for s in subtitles:
            output_subtitles.append([s.start.total_seconds(), s.end.total_seconds(), s.content])

        json.dump(output_subtitles, open(output_folder+str(df_merged.loc[i, 'uuid'])+".json", 'w'))
    except Exception as e:
        #print("Error: ", e)
        pass